# Assignment 1
Course: Mathematics of Generative Models \
Name: Md Faizal Karim \
Roll No: 2022121004

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import queue
import graphviz

plt.style.use('seaborn-v0_8')
np.__version__

'1.24.4'

## Problem 1

<img src= './soln1.jpeg' width='60%'>
Hence, we obtain p = 2/3

#### Simulating a Bernoulli random variable (p = 2/3) using numpy

In [2]:
x = np.random.binomial(n = 1, p = 2/3, size=30)
x = ['G' if i == 1 else 'B' for i in x]

print(x)

['G', 'G', 'B', 'G', 'G', 'B', 'G', 'B', 'B', 'G', 'G', 'G', 'B', 'B', 'G', 'G', 'B', 'G', 'B', 'B', 'B', 'B', 'G', 'G', 'G', 'G', 'G', 'G', 'B', 'B']


## Problem 2

In [5]:
def generate(n, samples, plot_cdf=False):
    """
    Input:
        n: n-sided die,
        samples: outcome of k trials
        plot_cdf: plot the cdf?
    Returns:
        out: a random sample from the calculated distribution
    """
    
    pmf = np.histogram(samples, bins=n)[0]/len(samples)
    cdf = np.cumsum(pmf)

    if plot_cdf:
        plt.title('CDF plot')
        plt.plot(cdf, label='cdf')
        plt.legend()
        plt.show()

    u = np.random.uniform(0, 1, 1)
    out = np.abs(cdf - u)
    out = np.where(np.abs(out - np.min(out)) < 1e-5)[0] + 1

    return out[np.random.choice(1, len(out))] if len(out) != 1 else out
    

In [6]:
samples = [1, 3, 2, 4, 2, 3, 5, 7, 3, 2]

for i in range(10):
    print(generate(n=6, samples=samples))

[3]
[2]
[1]
[2]
[3]
[5]
[6]
[1]
[4]
[2]


## Problem 3

In [174]:
samples = np.array([
    [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
    [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 0]])

class Node:
    def __init__(self, prob=None, index=0, is_leaf=False):
        self.prob = prob
        self.is_leaf = is_leaf
        self.left = None
        self.right = None
        self.index = index

# samples = samples[:, 0:8]
# samples, len(samples[0])

In [175]:
def fxn(node, i, condition):
    left_prob = len(np.where((samples[:, i] == 1) & (samples[:, i-1] == condition))[0])/len(np.where(samples[:, i-1] == condition)[0])
    right_prob = 1 - left_prob
    left_node, right_node = Node(prob=left_prob, index=i), Node(prob=right_prob, index=i)
    node.left = left_node
    node.right = right_node
    
    return left_node, right_node


def start(node):
    left_prob = len(np.where(samples[:, 0] == 1)[0])/len(samples)
    right_prob = len(np.where(samples[:, 0] == 0)[0])/len(samples)
    node.left = Node(prob=left_prob, index=1)
    node.right = Node(prob=right_prob, index=1)
    return node.left, node.right


node = Node()
q = queue.Queue()

left, right = start(node)
q.put(left)
q.put(right)

In [176]:
node.right.index

1

In [177]:

while q.qsize() != 0:
    for condition in [0, 1]:
        temp_node = q.get()
        if temp_node.index == len(samples[0])-1:
            continue
        
        if temp_node is None:
            continue
        
        left, right = fxn(temp_node, temp_node.index + 1, condition)
        q.put(left)
        q.put(right)
        

In [178]:
def create_graph(node, graph=None):
    if graph is None:
        graph = graphviz.Digraph(format='png', graph_attr={'rankdir': 'LR'})

    if node is not None:
        graph.node(str(id(node)), 
                   label=f"Prob: {node.prob:.4f}" if node.prob is not None else 'Start',
                   style='filled',
                   fillcolor='lightblue' if node.prob is None else 'white')

        if node.left is not None:
            graph.edge(str(id(node)), str(id(node.left)), label='1')
            create_graph(node.left, graph)

        if node.right is not None:
            graph.edge(str(id(node)), str(id(node.right)), label='0')
            create_graph(node.right, graph)

    return graph


graph = create_graph(node)


In [179]:
graph.render("tree_graph")

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.889175 to fit


'tree_graph.png'

In [97]:
node.left